In [14]:
# Copyright 2023 The Kubric Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [15]:
"""
Worker file for the Multi-Object Video (MOVi) C (and CC) datasets.
  * The number of objects is randomly chosen between
    --min_num_objects (3) and --max_num_objects (10)
  * The objects are randomly chosen from the Google Scanned Objects dataset

  * Background is an random HDRI from the HDRI Haven dataset,
    projected onto a Dome (half-sphere).
    The HDRI is also used for lighting the scene.
"""
import logging

import bpy
import kubric as kb
from kubric.simulator import PyBullet
from kubric.renderer import Blender
import numpy as np
from kubric import ArgumentParser
from argparse import Namespace
# local file
from kubric_helper import *

In [2]:
# --- Some configuration values
# the region in which to place objects [(min), (max)]
SPAWN_REGION   = [(-5, -5, 1), (5, 5, 5)]
VELOCITY_RANGE = [(-4., -4., 0.), (4., 4., 0.)]

FLAGS = Namespace(
    resolution  = 256, 
    frame_rate  = 12, 
    step_rate   = 240, 
    frame_start = 1, 
    frame_end   = 2, #24 
    logging_level = 'INFO', 
    seed = None, 
    scratch_dir = '/tmp/tmp82v1zpil', 
    job_dir     = 'output', 
    objects_split = 'train', 
    min_num_objects = 3, 
    max_num_objects = 10, 
    floor_friction    = 0.3, 
    floor_restitution = 0.5, 
    backgrounds_split = 'train', 
    camera = 'fixed_random', 
    max_camera_movement = 4.0, 
    kubasic_assets = 'gs://kubric-public/assets/KuBasic/KuBasic.json', 
    hdri_assets    = 'gs://kubric-public/assets/HDRI_haven/HDRI_haven.json', 
    gso_assets     = 'gs://kubric-public/assets/GSO/GSO.json', 
    save_state=False,
)

- https://polyhaven.com/hdris
- https://hdri-haven.com/category/indoor?page=2
- https://app.gazebosim.org/GoogleResearch

In [3]:
# --- Common setups & resources
scene, rng, output_dir, scratch_dir = kb.setup(FLAGS)

simulator = PyBullet(scene, scratch_dir)
renderer  = Blender(scene, scratch_dir, samples_per_pixel=64)

kubasic     = kb.AssetSource.from_manifest(FLAGS.kubasic_assets)
gso         = kb.AssetSource.from_manifest(FLAGS.gso_assets)
hdri_source = kb.AssetSource.from_manifest(FLAGS.hdri_assets)

INFO:kubric.utils:{ 'backgrounds_split': 'train',
  'camera': 'fixed_random',
  'floor_friction': 0.3,
  'floor_restitution': 0.5,
  'frame_end': 2,
  'frame_rate': 12,
  'frame_start': 1,
  'gso_assets': 'gs://kubric-public/assets/GSO/GSO.json',
  'hdri_assets': 'gs://kubric-public/assets/HDRI_haven/HDRI_haven.json',
  'job_dir': 'output',
  'kubasic_assets': 'gs://kubric-public/assets/KuBasic/KuBasic.json',
  'logging_level': 'INFO',
  'max_camera_movement': 4.0,
  'max_num_objects': 10,
  'min_num_objects': 3,
  'objects_split': 'train',
  'resolution': 256,
  'save_state': False,
  'scratch_dir': '/tmp/tmp82v1zpil',
  'seed': None,
  'step_rate': 240}
INFO:root:Deleting content of old scratch-dir: /tmp/tmp82v1zpil
INFO:root:Using scratch directory: /tmp/tmp82v1zpil
INFO:root:Using output directory: output
INFO:root:Created AssetSource 'KuBasic' with '15' assets at URI='gs://kubric-public/assets/KuBasic'
INFO:root:Created AssetSource 'GSO' with '1033' assets at URI='gs://kubric-publ

In [28]:
"small_cathedral".upper() in list(map(lambda x: x.upper(), train_backgrounds))

True

In [4]:
# --- Populate the scene
# background HDRI
train_backgrounds, test_backgrounds = hdri_source.get_test_split(fraction=0.1)
if FLAGS.backgrounds_split == "train":
  logging.info("Choosing one of the %d training backgrounds...", len(train_backgrounds))
  # RDM CHOICE
  hdri_id = rng.choice(train_backgrounds)
else:
  logging.info("Choosing one of the %d held-out backgrounds...", len(test_backgrounds))
  # RDM CHOICE
  hdri_id = rng.choice(test_backgrounds)
background_hdri = hdri_source.create(asset_id=hdri_id)
assert isinstance(background_hdri, kb.Texture)
logging.info("Using background %s", hdri_id)
scene.metadata["background"] = hdri_id
renderer._set_ambient_light_hdri(background_hdri.filename)

# Dome
dome = kubasic.create(asset_id="dome", name="dome",
                      friction=FLAGS.floor_friction,
                      restitution=FLAGS.floor_restitution,
                      static=True, background=True)
assert isinstance(dome, kb.FileBasedObject)

scene += dome

dome_blender = dome.linked_objects[renderer]
texture_node = dome_blender.data.materials[0].node_tree.nodes["Image Texture"]
texture_node.image = bpy.data.images.load(background_hdri.filename)

INFO:root:Choosing one of the 458 training backgrounds...
INFO:root:Using background hall_of_mammals


In [5]:
for a in scene.assets:
    print(a.name)

dome


In [6]:
# Camera
logging.info("Setting up the Camera...")
scene.camera = kb.PerspectiveCamera(focal_length=35., sensor_width=32)
if FLAGS.camera == "fixed_random":
  # RDM CHOICE
  scene.camera.position = kb.sample_point_in_half_sphere_shell(
      inner_radius=7., outer_radius=9., offset=0.1)
  scene.camera.look_at((0, 0, 0))
elif FLAGS.camera == "linear_movement":
  # RDM CHOICE
  camera_start, camera_end = get_linear_camera_motion_start_end(
      movement_speed=rng.uniform(low=0., high=FLAGS.max_camera_movement)
  )

  # linearly interpolate the camera position between these two points
  # while keeping it focused on the center of the scene
  # we start one frame early and end one frame late to ensure that
  # forward and backward flow are still consistent for the last and first frames
  for frame in range(FLAGS.frame_start - 1, FLAGS.frame_end + 2):
    interp = ((frame - FLAGS.frame_start + 1) /
              (FLAGS.frame_end - FLAGS.frame_start + 3))
    scene.camera.position = (interp * np.array(camera_start) +
                             (1 - interp) * np.array(camera_end))
    scene.camera.look_at((0, 0, 0))
    scene.camera.keyframe_insert("position", frame)
    scene.camera.keyframe_insert("quaternion", frame)


INFO:root:Setting up the Camera...


In [7]:
# Add random objects
train_split, test_split = gso.get_test_split(fraction=0.1)
if FLAGS.objects_split == "train":
  logging.info("Choosing one of the %d training objects...", len(train_split))
  active_split = train_split
else:
  logging.info("Choosing one of the %d held-out objects...", len(test_split))
  active_split = test_split

# RDM CHOICE
num_objects = rng.randint(FLAGS.min_num_objects,
                          FLAGS.max_num_objects+1)
logging.info("Randomly placing %d objects:", num_objects)

for i in range(num_objects):
  # RDM CHOICE
  obj = gso.create(asset_id=rng.choice(active_split))
  assert isinstance(obj, kb.FileBasedObject)

  # RDM CHOICE
  scale = rng.uniform(0.75, 3.0)
  obj.scale = scale / np.max(obj.bounds[1] - obj.bounds[0])
  obj.metadata["scale"] = scale
  scene += obj

  # RDM CHOICE???
  kb.move_until_no_overlap(obj, simulator, spawn_region=SPAWN_REGION, rng=rng)
  # initialize velocity randomly but biased towards center
  # RDM CHOICE
  obj.velocity = (rng.uniform(*VELOCITY_RANGE) -
                  [obj.position[0], obj.position[1], 0])
  logging.info("    Added %s at %s", obj.asset_id, obj.position)



INFO:root:Choosing one of the 930 training objects...
INFO:root:Randomly placing 7 objects:
INFO:root:    Added LEGO_Bricks_More_Creative_Suitcase at [-2.8712795 -2.1707032  1.9634649]
INFO:root:    Added Orbit_Bubblemint_Mini_Bottle_6_ct at [-1.6740121  3.5648253  4.1602254]
INFO:root:    Added Crayola_Bonus_64_Crayons at [-0.26495263  3.811788    2.382092  ]
INFO:root:    Added Office_Depot_HP_61Tricolor_Ink_Cartridge at [-4.108819  -2.3042092  1.918298 ]
INFO:root:    Added Lutein at [0.4963255 0.417453  3.8177557]
INFO:root:    Added Nintendo_Yoshi_Action_Figure at [-0.82072103 -2.9187102   2.5946877 ]
INFO:root:    Added Diet_Pepsi_Soda_Cola12_Pack_12_oz_Cans at [-0.35936362 -3.760623    1.929277  ]


In [8]:
if FLAGS.save_state:
  logging.info("Saving the simulator state to '%s' prior to the simulation.",
               output_dir / "scene.bullet")
  simulator.save_state(output_dir / "scene.bullet")

# Run dynamic objects simulation
logging.info("Running the simulation ...")
animation, collisions = simulator.run(frame_start=0,
                                      frame_end=scene.frame_end+1)

INFO:root:Running the simulation ...


In [10]:
# --- Rendering
if FLAGS.save_state:
  logging.info("Saving the renderer state to '%s' ",
               output_dir / "scene.blend")
  renderer.save_state(output_dir / "scene.blend")


logging.info("Rendering the scene ...")
data_stack = renderer.render()

# --- Postprocessing
kb.compute_visibility(data_stack["segmentation"], scene.assets)
visible_foreground_assets = [asset for asset in scene.foreground_assets
                             if np.max(asset.metadata["visibility"]) > 0]
visible_foreground_assets = sorted(  # sort assets by their visibility
    visible_foreground_assets,
    key=lambda asset: np.sum(asset.metadata["visibility"]),
    reverse=True)

data_stack["segmentation"] = kb.adjust_segmentation_idxs(
    data_stack["segmentation"],
    scene.assets,
    visible_foreground_assets)
scene.metadata["num_instances"] = len(visible_foreground_assets)

# Save to image files
kb.write_image_dict(data_stack, output_dir)
kb.post_processing.compute_bboxes(data_stack["segmentation"],
                                  visible_foreground_assets)

# --- Metadata
logging.info("Collecting and storing metadata for each object.")
metadata = {
    "flags": vars(FLAGS),
    "metadata": kb.get_scene_metadata(scene),
    "camera": kb.get_camera_info(scene.camera),
    "instances": kb.get_instance_info(scene, visible_foreground_assets),
}
kb.write_json(filename=output_dir / "metadata.json", data=metadata)
kb.write_json(filename=output_dir / "events.json", data={
    "collisions":  kb.process_collisions(
        collisions, scene, assets_subset=visible_foreground_assets),
})

kb.done()


INFO:root:Rendering the scene ...
INFO:kubric.renderer.blender:Using scratch rendering folder: '/tmp/tmp82v1zpil'


Fra:1 Mem:55.66M (Peak 759.72M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | FileBasedObject.006
Fra:1 Mem:55.74M (Peak 759.72M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | FileBasedObject.003
Fra:1 Mem:56.74M (Peak 759.72M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | FileBasedObject.005
Fra:1 Mem:64.89M (Peak 759.72M) | Time:00:00.04 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Initializing
Fra:1 Mem:52.11M (Peak 759.72M) | Time:00:00.04 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Waiting for render to start
Fra:1 Mem:52.11M (Peak 759.72M) | Time:00:00.04 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Loading render kernels (may take a few minutes the first time)
Fra:1 Mem:52.11M (Peak 759.72M) | Time:00:00.04 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Updating Scene
Fra:1 Mem:52.11M (Peak 759.72M) | Time:00:00.04 | Mem:0.00M, Peak:0.00M | Scene, View Layer | 

INFO:kubric.renderer.blender:Rendered frame '/tmp/tmp82v1zpil/images/frame_0001.png'


Saved: '/tmp/tmp82v1zpil/images/frame_0001.png'
 Time: 00:04.01 (Saving: 00:00.31)

Fra:2 Mem:55.91M (Peak 55.91M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | FileBasedObject
Fra:2 Mem:55.91M (Peak 55.91M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | FileBasedObject.006
Fra:2 Mem:56.93M (Peak 56.93M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | FileBasedObject.005
Fra:2 Mem:65.14M (Peak 66.63M) | Time:00:00.03 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Initializing
Fra:2 Mem:52.36M (Peak 66.63M) | Time:00:00.03 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Waiting for render to start
Fra:2 Mem:52.36M (Peak 66.63M) | Time:00:00.03 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Loading render kernels (may take a few minutes the first time)
Fra:2 Mem:52.36M (Peak 66.63M) | Time:00:00.03 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Updating Scene
Fra:2 Mem:52.36M (Pea

INFO:kubric.renderer.blender:Rendered frame '/tmp/tmp82v1zpil/images/frame_0002.png'


Fra:2 Mem:265.69M (Peak 902.84M) | Time:00:03.88 | Mem:215.64M, Peak:407.61M | Scene, AuxOutputs | Updating Integrator
Fra:2 Mem:266.96M (Peak 902.84M) | Time:00:03.89 | Mem:216.91M, Peak:407.61M | Scene, AuxOutputs | Updating Film
Fra:2 Mem:266.96M (Peak 902.84M) | Time:00:03.89 | Mem:216.66M, Peak:407.61M | Scene, AuxOutputs | Updating Lookup Tables
Fra:2 Mem:266.96M (Peak 902.84M) | Time:00:03.89 | Mem:216.91M, Peak:407.61M | Scene, AuxOutputs | Updating Baking
Fra:2 Mem:266.96M (Peak 902.84M) | Time:00:03.89 | Mem:216.91M, Peak:407.61M | Scene, AuxOutputs | Updating Device | Writing constant memory
Fra:2 Mem:266.96M (Peak 902.84M) | Time:00:03.89 | Mem:216.91M, Peak:407.61M | Scene, AuxOutputs | Updating Device | Writing constant memory | Compiling render kernels
Fra:2 Mem:266.96M (Peak 902.84M) | Time:00:03.89 | Mem:216.91M, Peak:407.61M | Scene, AuxOutputs | Updating Device | Writing constant memory
Fra:2 Mem:266.96M (Peak 902.84M) | Time:00:03.89 | Mem:216.91M, Peak:407.61M | Sc

INFO:root:Writing to 'output/rgba_00000.png'
INFO:root:Writing to 'output/rgba_00001.png'
INFO:root:Writing to 'output/backward_flow_00000.png'
INFO:root:Writing to 'output/backward_flow_00001.png'
INFO:root:Writing to 'output/data_ranges.json'
INFO:root:Writing to 'output/forward_flow_00000.png'
INFO:root:Writing to 'output/forward_flow_00001.png'
INFO:root:Writing to 'output/data_ranges.json'
INFO:root:Writing to 'output/normal_00001.png'
INFO:root:Writing to 'output/normal_00000.png'
INFO:root:Writing to 'output/object_coordinates_00000.png'
INFO:root:Writing to 'output/object_coordinates_00001.png'
INFO:root:Writing to 'output/segmentation_00000.png'
INFO:root:Writing to 'output/segmentation_00001.png'
INFO:root:Collecting and storing metadata for each object.
INFO:root:Writing to 'output/metadata.json'
INFO:root:Writing to 'output/events.json'
INFO:root:Done!
